In [0]:
#!pip install python-slugify
#!pip install loguru
#!pip install openpyxl

!pip install --upgrade pip
!pip install python-slugify loguru openpyxl scikit-learn joblib shap imbalanced-learn xgboost

DEPRECATION: Using the pkg_resources metadata backend is deprecated. pip 26.3 will enforce this behaviour change. A possible replacement is to use the default importlib.metadata backend, by unsetting the _PIP_USE_IMPORTLIB_METADATA environment variable. Discussion can be found at https://github.com/pypa/pip/issues/13317
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
DEPRECATION: Using the pkg_resources metadata backend is deprecated. pip 26.3 will enforce this behaviour change. A possible replacement is to use the default importlib.metadata backend, by unsetting the _PIP_USE_IMPORTLIB_METADATA environment variable. Discussion can be found at https://github.com/pypa/pip/issues/13317
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/253.9 MB ? eta -:--:--
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/253.9 MB 341.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 128.5/253.9 MB 343.6 MB/s eta 0:00:01


In [0]:

import pandas as pd
import numpy as np
import joblib
from uuid import uuid4
from abc import ABC, abstractmethod
import json
import io
import os
from requests.structures import CaseInsensitiveDict
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from pyspark.sql.functions import split, trim, col, substring
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from loguru import logger
from uuid import uuid4
from abc import ABC, abstractmethod
import json
import io
import os

import pandas as pd
import numpy as np
import pyspark.pandas as ps
import requests
from requests.structures import CaseInsensitiveDict
from slugify import slugify
from functools import partial
from loguru import logger
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from pyspark.sql.functions import split, trim, col, substring
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
import joblib

from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils


In [0]:
## Permite a conexão com Sharepoint
spark = SparkSession.builder.getOrCreate()
delta = SparkSession.builder.appName("").getOrCreate()
dbutils = DBUtils(spark)

secret_dict = {
'Databricks':'key-trdg-bricks-token',
'Sharepoint':'key-trdg-client-secret',
'FTP':'key-trdg-ftp',
'SAS':'key-trdg-sas',
}

def get_secret(which: str):
    try:
        return dbutils.secrets.get(scope="keyvault", key=secret_dict[which])
    except:
        print('The secrets available are: Databricks[Token], Sharepoint[Token], FTP[Password] and SAS[User and Password].')


URL_TOKEN = 'https://login.microsoftonline.com/24090322-b104-494d-a1d3-662da14cddd4/oauth2/v2.0/token'
URL_MSG = 'https://graph.microsoft.com/v1.0/sites/'

BLOB_PATH = 'abfss://sandbox@adltrdgwestus.dfs.core.windows.net/'

#spark.conf.set(
#    "fs.azure.account.key.adltrdgwestus.dfs.core.windows.net",
#    dbutils.secrets.get(scope="scope-keyvault-prd", key="secret-databricks-trdg-prd"))


class Sharepoint(ABC):
    """Class that implements the logic to extract and load
    datasets from the Sharepoint to the Databricks.
    """
    def __init__(self, path_file: str, name_site: str = 'DadosBI', host: str = 'achelaboratorios.sharepoint.com'):
        self.path_file = path_file 
        self.name_site = name_site 
        self.host = host

        self.headers = self.get_bearer_token()
        self.data = self.download_file()

    
    def get_bearer_token(self) -> dict:
        body = {'grant_type': 'client_credentials', 
                'client_id': '01c78346-6928-48c4-8cd6-d0ef71ec7021', 
                'client_secret': get_secret(which='Sharepoint'),
                'scope': 'https://graph.microsoft.com/.default'
                }
        
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        headers["Authorization"] = f"Bearer {json.loads(requests.post(URL_TOKEN, data=body).text)['access_token']}"

        return headers


    def get_ids(self, is_file=True) -> str:
        id_site = (requests
                   .get(f'{URL_MSG}/{self.host}:/sites/'+self.name_site+'?$select=id', 
                        headers=self.headers)
                   .json()['id']
                   .split(',')
                   [1]
                   )
        id_drives = (requests
                    .get(URL_MSG+id_site+f"/drive", 
                                 headers=self.headers)
                    .json()
                    ['id']
                    )
        if is_file:
            url_content = URL_MSG+id_site+'/drives/'+id_drives+f'/root:/{self.path_file}:/content'
        else: 
            url_content = URL_MSG+id_site+'/drives/'+id_drives+f'/root:/{self.path_file}:/children'
        return url_content
    

    def download_file(self) -> dict:
        if '.' in self.path_file:
            url_content = self.get_ids()
        else:
            url_content = self.get_ids(is_file=False)
        return (requests.get(url_content, headers=self.headers)).content
    
    def ls(self) -> list:
        data_dict = json.loads(self.data)
        list_files = []
        for files in data_dict['value']:
            list_files.append(files['name'])
        return list_files
    
    def read_file(self, enconding:str = 'utf-8', **params) -> pd.DataFrame:
        type_file = self.path_file.split('.')[1]
        if type_file in ['txt', 'csv']:
            self.read_data = pd.read_csv(io.StringIO(self.data.decode(enconding)), **params)
            return self
        elif type_file in ['xls', 'xlsx']:
            self.read_data = pd.read_excel(self.data, **params)
            return self
        else:
            print('Este tipo de arquivo não está implementado!')
    

    def to_dataframe(self) -> pd.DataFrame:
        return self.read_data
    

    def export_blob_storage_csv(self, path:str, delimiter:str = ';', encoding:str = "UTF-8") -> str:
        ps.from_pandas(self.read_data).to_spark() \
        .coalesce(1) \
            .write \
            .mode("overwrite") \
            .option("header", "true") \
            .option("delimiter", delimiter) \
            .option("encoding",encoding) \
            .csv(BLOB_PATH + path)


    def slugify_columns(self, columns):
        slug = partial(slugify, separator="_")
        return [slug(column.replace("%", "percent").replace("+", "_")) for column in columns]


    def export_blob_storage_parquet(self, path:str, compression:str = 'snappy') -> str:
        df = self.read_data
        df.columns = self.slugify_columns(df.columns)
        ps.from_pandas(df).to_spark() \
        .coalesce(1) \
            .write \
            .mode("overwrite") \
            .option("compression", compression) \
            .parquet(BLOB_PATH + path)
    
    
    def export_datalake(self, path) -> str:
        ps.from_pandas(self.read_data).to_spark() \
         .write \
         .mode("overwrite") \
         .option("overwriteSchema", "true") \
         .saveAsTable(path)


In [0]:

def preprocess_data(df, fit_scaler=True, scaler=None):
    logger.info("Iniciando pré-processamento...")
    
    df = df.dropna()
    features = [
        'SEGMENTO', 'PRIMEIRA_COMPRA', 
        'PRIMEIRA_COMPRA_CRM_LISTA_DE_BLOQUEIO_perc',
        'PRIMEIRA_COMPRA_CPF_INATIVO_perc', 
        'QTD_MARCA_AVG', 'QTD_MARCA_qt50'
    ]
    
    X = df[features].copy()
    Y = df['Desvio'].map({'N': 0, 'S': 1}).astype(int)
    
    X['SEGMENTO'] = LabelEncoder().fit_transform(X['SEGMENTO'])
    
    if fit_scaler:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = scaler.transform(X)
        
    logger.success("Pré-processamento finalizado.")
    return X_scaled, Y.values, scaler


In [0]:

def train_model(X, Y):
    logger.info("Treinando modelo...")
    smote = SMOTE(random_state=42)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)
    
    model = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    
    model.fit(X_resampled, Y_resampled)
    logger.success("Modelo treinado.")
    return model


In [0]:

def validate_model(model, X, Y):
    logger.info("Validando modelo com cross-validation...")
    skf = StratifiedKFold(n_splits=5)
    scores = cross_val_score(model, X, Y, cv=skf, scoring='accuracy')
    logger.success(f"Acurácia média: {scores.mean():.4f}")
    return scores


In [0]:

def evaluate_model(model, X_test, Y_test):
    logger.info("Avaliando modelo...")
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test, Y_pred))
    print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")
    print(f"ROC AUC: {roc_auc_score(Y_test, Y_pred):.4f}")
    cm = confusion_matrix(Y_test, Y_pred)
    print("Matriz de Confusão:\n", cm)


In [0]:

def save_model(model, scaler, model_path='modelo_xgb.pkl', scaler_path='scaler.pkl'):
    joblib.dump(model, model_path)
    joblib.dump(scaler, scaler_path)
    logger.success("Modelo e scaler salvos.")


In [0]:

def predict_new_data(df_new, model, scaler):
    logger.info("Realizando predição em novos dados...")
    X_new, _, _ = preprocess_data(df_new, fit_scaler=False, scaler=scaler)
    preds = model.predict(X_new)
    df_new['Desvio_predito'] = preds
    return df_new


In [0]:

# 1. Ler os dados
df = Sharepoint('General/Arquivo_Databricks/Aprendizagem_fraude_teste.xlsx').read_file(engine = 'openpyxl').to_dataframe()

# 2. Pré-processar
X, Y, scaler = preprocess_data(df)

# 3. Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# 4. Treinar
modelo_xgb = train_model(X_train, y_train)

# 5. Validar
validate_model(modelo_xgb, X, Y)

# 6. Avaliar
evaluate_model(modelo_xgb, X_test, y_test)

# 7. Salvar
save_model(modelo_xgb, scaler)


2025-05-29 14:42:11.926 | INFO     | __main__:preprocess_data:2 - Iniciando pré-processamento...
2025-05-29 14:42:11.933 | SUCCESS  | __main__:preprocess_data:23 - Pré-processamento finalizado.
2025-05-29 14:42:11.936 | INFO     | __main__:train_model:2 - Treinando modelo...


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

/local_disk0/.ephemeral_nfs/envs/pythonEnv-4f7ecd26-5ece-45e8-82fa-cd01e3081d52/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:42:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025-05-29 14:42:26.298 | SUCCESS  | __main__:train_model:16 - Modelo treinado.
2025-05-29 14:42:26.299 | INFO     | __main__:validate_model:2 - Validando modelo com cross-validation...
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4f7ecd26-5ece-45e8-82fa-cd01e3081d52/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:42:26] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4f7ecd26-5ece-45e8-82fa-cd01e3081d52/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:42:26] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_en

              precision    recall  f1-score   support

           0       0.85      0.94      0.89        31
           1       0.67      0.44      0.53         9

    accuracy                           0.82        40
   macro avg       0.76      0.69      0.71        40
weighted avg       0.81      0.82      0.81        40

Accuracy: 0.8250
ROC AUC: 0.6900
Matriz de Confusão:
 [[29  2]
 [ 5  4]]


2025-05-29 14:42:48.382 | SUCCESS  | __main__:save_model:4 - Modelo e scaler salvos.
